
## Sesión 4.Introducción al Aprendizaje por Refuerzo en entornos discretos
#### Inteligencia Artificial 2024 / 25
##### Belén Díaz Agudo 

## Librería OpenAI Gym  y Gymnasium

[OpenAI Gym](https://www.gymlibrary.dev/) (Brockman et. al., 2016) es una librería de OpenAI que ofrece entornos y una interfaz estándar con la cuál podemos diseñar y probar agentes. Su objetivo es proveer benchmarks unificados para ver el desempeño de algoritmos con el mismo entorno y así poder saber con facilidad cómo es su desempeño comparado con los demás. La librería ha evolucionado a una nueva versión, [gymnasium](https://gymnasium.farama.org/index.html) que es compatible con los entornos anteriores realizados en gym. 
Aunque la práctica también se puede hacer con la librería gym utilizaremos gymnasium que es una librería más moderna. Necesitaremos instalar las librerías numpy, gymnasium y pygame. 


La interfaz principal de los entornos (environments) de gym es la interfaz __env__ con 5 métodos principales:

* ```reset(self)``` : Reinicia el estado del entorno, a su estado inicial, devolviendo una observación de dicho estado.

* ```step(self, action)``` : "Avanza" un timestep. Devuelve: ```observation, reward, done, info```.

* ```render(self)``` : Muestra en pantalla el entorno.

* ```close(self)``` : Finaliza con la instancia del agente.

* ```seed(self)``` : Establece la semilla aleatoria del generador de números aleatorios del presente entorno.
To get reproducible sampling of actions, a seed can be set with env.action_space.seed(123).

Puedes consultar la documentación de los entornos en https://gymnasium.farama.org/api/env/#gymnasium.Env

Cada entorno posee los siguientes tres atributos principales:

* ```action_space``` : El objeto de tipo Space correspondiente al espacio de acciones válidas.

* ```observation_space``` : El objeto de tipo Space correspondiente a todos los rangos posibles de observaciones.

* ```reward_range``` : Tupla que contiene los valores mínimo y máximo de recompensa posible.

## Ejemplo 1. Taxi

En este ejemplo hay 4 ubicaciones (etiquetadas con letras o colores diferentes) y nuestro trabajo es recoger al pasajero en una ubicación de origen y dejarlo en otra posición destino. El taxi solo puede coger y dejar pasajeros en las 4 posiciones marcadas.

Como recompensa recibimos +20 puntos por dejar al pasajero con éxito y perderemos 1 punto por cada paso de tiempo. También hay una penalización de 10 puntos por acciones ilegales de recoger y dejar en ubicaciones no válidas.

Revisa la documentación del entorno en Gymnasium [https://gymnasium.farama.org/environments/toy_text/taxi/](https://gymnasium.farama.org/environments/toy_text/taxi/)

In [ ]:
import gymnasium as gym
#inicializamos en entorno gym con el problema del taxi y visualización gráfica.
env = gym.make("Taxi-v3", render_mode="human")

Vamos a observar algunos datos del entorno, en concreto cuantos estados y acciones tenemos.

In [ ]:
print("Tamaño de espacio de estados", env.observation_space)
print("Estado aleatorio", env.observation_space.sample())
size_estados = env.observation_space.n
print("Hay", size_estados, " estados posibles.")
print("Acciones posibles", env.action_space.n)
print("Acción aleatoria", env.action_space.sample())
size_acciones = env.action_space.n
print("Hay", size_acciones, " acciones posibles.")


Tamaño de espacio de estados Discrete(500)
Estado aleatorio 369
Hay 500  estados posibles.
Acciones posibles 6
Acción aleatoria 5
Hay 6  acciones posibles.


In [ ]:
# Podemos crear un estado aleatorio con reset y visualizarlo con render
env.reset()
env.render()
# En este ejemplo, la visualización "humana" suele dar problemas y para este ejemplo vamos a usar una visualizacion más sencilla (ansi). 
# Se puede configurar el tipo de visualización en el make. 

In [ ]:
# Cuando termines de trabajar es importante cerrar el entorno. Puedes volver a crearlo con make.
env.close()

In [ ]:
# la visualizacion humana está bien, pero si queremos hacer un experimento con muchos episodios, es mejor no visualizarlo. 
# En este ejemplo vamos a usar una visualización en modo texto para poder
# guardar los tableros en un diccionario para visualizarlo despues del entrenamiento. 
# Usaremos en el make una de las opciones adicioanles de visualización, en concreto "ansi" 
env = gym.make("Taxi-v3", render_mode="ansi")
env.reset()
print(env.render())
# Recuerda que debers cerrar el entorno con problema.close() al finalizar. Ahora no vamos a cerrarlo de momento porque seguimos trabajando. 

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




- El cuadrado representa el taxi. Es amarillo sin pasajero y verde cuando lleva pasajero.
- La marca ("|") representa una pared que el taxi no puede cruzar.
- R, G, Y, B son las posibles ubicaciones de recogida y destino. La letra que esté coloreada en azul representa la ubicación actual de recogida de pasajeros y la letra rosa es el destino actual.

In [ ]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


Lo primero siempre es entender el entorno. 

El Action Space tiene tamaño 6 (hay 6 acciones) y el State Space tiene tamaño 500 (500 estados). El algoritmo RL no necesita más información que estas dos cosas. Todo lo que necesitamos es una forma de identificar un estado de forma única asignando un número único a cada estado posible, y RL aprende a elegir un número de la acción más conveniente en ese estado. Las 6 acciones se numeran de 0 a 5 donde:
   - 0 = sur
   - 1 = norte
   - 2 = este
   - 3 = oeste
   - 4 = recoger
   - 5 = dejar

¿Entiendes por qué hay 500 estados diferentes?  
Los 500 estados corresponden a una codificación de la ubicación del taxi (fila 0..4, columna 0..4), la ubicación inicial, la ubicación actual del pasajero (5 valores posibles: 4 posiciones R(0), G(1), Y(2), B(3) o en el taxi (4)) y la ubicación de destino (4 valores posibles R(0), G(1), Y(2), B(3)): 5x5x5x4 = 500

La coordenadas (0,0) es la esquina superior izquierda del tablero.

El aprendizaje por refuerzo aprenderá un mapeo de los estados con las acciones óptimas. Este aprendizaje se guarda en la matriz Q. 
Durante el aprendizaje realizaremos un proceso de _exploración_, es decir, el agente explora el entorno y toma acciones basadas en las recompensas definidas en el entorno. 

La acción óptima para cada estado es la acción que tiene la mayor recompensa acumulativa a largo plazo según la fórmula vista para el Q-learning.


In [ ]:
# Observa y ejecuta este fragmento de código en el que se intenta resolver el problema realizando 
# acciones aleatorias sobre el tablero del taxi. 
# Lo normal es que no resuelva el problema :) 
# Se guardan los estados en un diccionario. Puedes ver luego la animación en la siguiente celda.

epochs = 0
penalties, reward = 0, 0
frames = [] # for animation
done = False
while not done:
    action = env.action_space.sample()
    state, reward, done, info, _ = env.step(action)
    if reward == -10:
        penalties += 1
    # Guardamos cada frame en un diccionario y despues veremos la animación. 
    frames.append({
        'frame': env.render(),
        'state': state,
        'action': action,
        'reward': reward
        }
    )
    epochs += 1
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 2643
Penalties incurred: 833


In [ ]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames,time=.2):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        #print(frame['frame'].getvalue())
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(time)

In [ ]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)

Timestep: 12
State: 254
Action: 3
Reward: -1


KeyboardInterrupt: 

En esta versión el agente va totalmente a ciegas __sin aprender__. Utiliza miles de pasos de tiempo y realiza muchos drop-offs incorrectos para entregar un solo pasajero al destino correcto (cuando acierte de casualidad). ¿Ha tardado mucho en acertar de casualidad?  
Vamos a resolverlo con Aprendizaje por refuerzo que empieza haciendo ciclos a ciegas pero a la vez aprendiendo de la experiencia. Realizará pasos de forma aleatoria y, cuando "por casualidad" acertamos, el agente guarda en la memoria (en forma de recompensa) qué acción fue la mejor para cada estado. Así en el futuro elegirá esa acción. Los éxitos se propagan hacia atrás en la cadena de acciones usando la fórmula del Q-Learning.

In [ ]:
env.close()

### Aprendizaje por refuerzo

Vamos a utilizar el algoritmo de Q-learning que hemos visto en clase y que le dará a nuestro agente algo de memoria.
Básicamente, Q-learning permite al agente utilizar las recompensas del entorno para aprender, con el tiempo, la mejor acción a realizar en un estado determinado.
En nuestro entorno de Taxi, tenemos la tabla de recompensas, P, de la que el agente aprenderá. Lo hace buscando recibir una recompensa por realizar una acción en el estado actual y luego actualizar un valor Q para recordar si esa acción fue beneficiosa.
Los valores almacenados en la tabla Q se denominan valores Q y se asignan a una combinación (estado, acción).
Un valor Q para una combinación de acción de estado particular es representativo de la "calidad" de una acción tomada desde ese estado. Mejores valores Q implican mejores posibilidades de obtener mayores recompensas.
Por ejemplo, si el taxi se enfrenta a un estado que incluye a un pasajero en su ubicación actual, es muy probable que el valor Q para la recogida sea más alto en comparación con otras acciones, como la bajada o el norte.
Los valores Q se inicializan a un valor arbitrario y, a medida que el agente se expone al entorno y recibe diferentes recompensas al ejecutar diferentes acciones, los valores Q se actualizan mediante la ecuación:

Q (estado, acción) ← (1 − α) Q (estado, acción) + α (recompensa + γ maxa Q (siguiente estado, todas las acciones))

Dónde:
- α (alfa) es la tasa de aprendizaje (0 <α≤1) - Al igual que en los entornos de aprendizaje supervisado, αα es la medida en que nuestros valores Q se actualizan en cada iteración.
- γ (gamma) es el factor de descuento (0≤γ≤1) - determina cuánta importancia queremos dar a las recompensas futuras. Un valor alto para el factor de descuento (cercano a 1) captura la recompensa efectiva a largo plazo, mientras que un factor de descuento de 0 hace que nuestro agente considere solo la recompensa inmediata.

En la fórmula anterior estamos asignando (←), o actualizando, el valor Q del estado actual y la acción del agente tomando primero un peso (1 − α) del antiguo valor Q y luego agregando el valor aprendido. El valor aprendido es una combinación de la recompensa por realizar la acción actual en el estado actual y la recompensa máxima descontada del siguiente estado en el que estaremos una vez que realicemos la acción actual.
Básicamente, estamos aprendiendo la acción adecuada a tomar en el estado actual al observar la recompensa por la combinación estado / acción actual y las recompensas máximas para el siguiente estado. Esto eventualmente hará que nuestro taxi considere la ruta con las mejores recompensas.
El valor Q de un par estado-acción es la suma de la recompensa instantánea y la recompensa futura descontada (del estado resultante). La forma en que almacenamos los valores Q para cada estado y acción es a través de una tabla Q

La tabla Q es una matriz donde tenemos una fila para cada estado (500) y una columna para cada acción (6). Primero se inicializa a 0 y luego los valores se actualizan después del entrenamiento. Tenga en cuenta que la Q-table tiene las mismas dimensiones que la mesa de recompensas, pero tiene un propósito completamente diferente.

En la siguiente figura los valores de la Q-Table se inicializan a cero y se van actualizando durante el aprendizaje.  Los valores optimizan el recorrido del agente a traves del entorno buscando las máximas recompensas. 

### Resumen del proceso de Q-Learning

    • Inicializar la tabla Q a todo ceros.
    • Comenzar a explorar acciones: para cada estado, seleccione cualquiera de las posibles acciones para el estado actual (S).
    • Ir al siguiente estado (S ') como resultado de esa acción (a).
    • Para todas las acciones posibles del estado (S '), seleccione la que tenga el valor Q más alto.
    • Actualizar los valores de la tabla Q utilizando la ecuación.
    • Establecer el siguiente estado como el estado actual.
    • Si se alcanza el estado objetivo, finalizar y repetir el proceso.

### Explotación de valores aprendidos y ϵ épsilon learning

Después de una fase de exploración aleatoria de acciones, los valores Q tienden a converger sirviendo a nuestro agente como una función de valor de acción que puede explotar para elegir la acción mejor para un estado dado.
Existe una compensación entre exploración (elegir una acción aleatoria) y explotación (elegir acciones basadas en valores Q ya aprendidos). Queremos evitar que la acción siga siempre la misma ruta y posiblemente se sobreajuste, por lo que introduciremos otro parámetro llamado ϵ "épsilon" para atender esto durante el entrenamiento.
En lugar de simplemente seleccionar la acción de valor Q mejor aprendida, a veces preferimos explorar más el espacio de acción. Un valor de épsilon más bajo da como resultado episodios con más penalizaciones (en promedio), lo cual es obvio porque estamos explorando y tomando decisiones al azar.

In [ ]:
# Primero se inicializa la Q-table a 500×6500×6 matrix of zeros:
env = gym.make("Taxi-v3")
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

Ejecutar el aprendizaje es un proceso lento pero se realiza una única vez. 
Una vez entrenado, podemos resolver cualquier problema de este entorno simplemente consultando la tabla y eligiendo la acción que maximiza la recompensa en cada paso (explotación). 
El algoritmo de entrenamiento que actualizará esta Q-table a medida que el agente explora el entorno durante miles de episodios. 

En la primera parte del bucle (while not done) decidimos si elegir una acción aleatoria o explotar los valores Q ya calculados. Esto se hace simplemente usando el valor épsilon y comparándolo con la función random.uniform (0, 1), que devuelve un número arbitrario entre 0 y 1.
Ejecutamos la acción elegida en el entorno para obtener el next_state y la recompensa por realizar la acción. Después de eso, calculamos el valor Q máximo para las acciones correspondientes al next_state, y con eso, podemos actualizar fácilmente nuestro valor Q al new_q_value:

In [ ]:
state, _ = env.reset()
print(state  )

In [ ]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1
episodes = 100000

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, episodes+1):
    state,_ = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info, _ = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")


In [ ]:
# La tabla Q empezó inicializada a 0. Ahora ha cambiado despues de 100000 episodios. Vamos a ver cuales son los Q-values aprendidos en un estado de ejemplo. 
q_table[328]
# será aproximadamente valores aprendidos similares a:  array([ -2.40177682,  -2.27325184,  -2.38277076,  -2.35245849, -11.14888317, -10.43479034])  (no tiene por que ser exactamente igual)
# Tenemos el valor aprendido para las 6 acciones. El máximo lo encontramos en la segunda opción (norte) que es lo que ha aprendido. 

### Evaluar el comportamiento del agente despues del proceso de Q-learning 

Una vez entrenado el agente vamos a evaluar si el comportamiento de nuestro agente es adecuado con los valores que tenemos en la tabla Q. Ahora no se eligen nunca acciones aleatorias sino que siempre estaremos en __explotación__: la siguiente acción siempre se selecciona utilizando el mejor valor Q:

In [ ]:
total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state,_ = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[int(state)])
        state, reward, done, info, _ = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")


Podemos ver en la evaluación que el comportamiento del agente ha mejorado significativamente y _no hay penalizaciones_, lo que significa que realizó las acciones correctas de recogida / devolución con 100 pasajeros diferentes. Repasa lo que se considera penalización en este problema. 

Comparando los dos agentes vemos como aunque con Q-learning el agente comete errores inicialmente durante la exploración, una vez que ha explorado lo suficiente (visto la mayoría de los estados), puede actuar sabiamente maximizando las recompensas haciendo movimientos inteligentes. 

Veamos cuánto mejor es nuestra solución de Q-learning en comparación con el agente que realiza movimientos aleatorios.

Se puede evaluar al agente definiendo métricas. Las métricas pueden ser diferentes para cada problema. En este caso se proponen las siguientes métricas:

    • Número medio de penalizaciones por episodio: Cuanto menor sea el número, mejor será el desempeño de nuestro agente. Idealmente, nos gustaría que esta métrica fuera cero o muy cercana a cero.

    • Número promedio de pasos de tiempo por viaje: también queremos un número pequeño de pasos de tiempo por episodio, ya que queremos que nuestro agente dé pasos mínimos (es decir, el camino más corto) para llegar al destino.

    • Promedio de recompensas por movimiento: cuanto mayor sea la recompensa, significa que el agente está haciendo lo correcto. Es por eso que decidir las recompensas es una parte crucial del aprendizaje por refuerzo. En nuestro caso, dado que tanto los tiempos como las penalizaciones se recompensan negativamente, una recompensa promedio más alta significaría que el agente llega al destino lo más rápido posible con la menor cantidad de penalizaciones.


    Average rewards per move  -3.9012092102214075	0.6962843295638126
    Average number of penalties per episode	920.45	0.0
    Average number of timesteps per trip	2848.14	12.38

Estas métricas se calcularon en más de 100 episodios. 
Y como muestran los resultados, podemos concluir que nuestro agente de Q-learning tiene un buen comportamiento. 


In [ ]:
env.close()

### Hiperparámetros y optimizaciones

Los valores de `alpha`,` gamma` y `epsilon` que hemos utilizado han sido elegidos por intuición, prueba y error pero hay mejores formas de obtener buenos valores. Idealmente, los tres deberían disminuir con el tiempo porque a medida que el agente continúa aprendiendo, en realidad construye antecedentes más válidos y duraderos. 

En tus entrenamientos se pueden usar hiperparámetros variables que modifican dinámicamente su valor en función de la etapa del entrenamiento.

    • α: (la tasa de aprendizaje) debería disminuir a medida que continúa adquiriendo una base de conocimientos cada vez mayor.
    • γ: a medida que se acerca cada vez más al valor límite, su preferencia por la recompensa a corto plazo debería aumentar, ya que no estará el tiempo suficiente para obtener la recompensa a largo plazo, lo que significa que su gamma debería disminuir.
    • ϵ: a medida que desarrollamos nuestra estrategia, tenemos menos necesidad de exploración y más explotación para obtener más utilidad de nuestra política, por lo que a medida que aumentan los ensayos, épsilon debería disminuir.

In [ ]:
import random
from IPython.display import clear_output

def trainAgent(q_table,alpha,gamma,epsilon,iterations=100000):
    """Training the agent"""
    # For plotting metrics
    all_epochs = []
    all_penalties = []

    for i in range(1, iterations+1):
        state, _ = env.reset()

        epochs, penalties, reward, = 0, 0, 0
        done = False

        while not done:
            if random.uniform(0, 1) < epsilon:
                action = env.action_space.sample() # Explore action space
            else:
                action = np.argmax(q_table[state]) # Exploit learned values

            next_state, reward, done, info, _ = env.step(action) 

            old_value = q_table[state, action]
            next_max = np.max(q_table[next_state])

            new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            q_table[state, action] = new_value

            if reward == -10:
                penalties += 1

            state = next_state
            epochs += 1

#       if i % 100 == 0:
#            clear_output(wait=True)
#            print(f"Episode: {i}")

    print("Training finished.\n")

In [ ]:
def solveTaxi(q_table,showF=False,time=.2):
    """Solves the Taxi Problem"""
     
    state,_= env.reset()
    epochs = 0
    penalties, reward = 0, 0
    reward = 0
    frames = [] # for animation
    done = False

    while not done:
        action = np.argmax(q_table[int(state)])
        # Put each rendered frame into dict for animation
        frames.append({
            'frame': env.render(),
            'state': state,
            'action': action,
            'reward': reward
            }
        )
        state, reward, done, info, _ = env.step(action)

        if reward == -10:
            penalties += 1

        

        epochs += 1
        
        if epochs >=1000:
            done = True


    print("Timesteps taken: {}".format(epochs))
    print("Penalties incurred: {}".format(penalties))
    
    if (showF):
        print_frames(frames,time)        
    env.close()

In [3]:
import numpy as np

In [ ]:
%%time
env = gym.make("Taxi-v3",render_mode="ansi")

q_table = np.zeros([env.observation_space.n, env.action_space.n])
trainAgent(q_table,0.1,0.6,0.1,1000)

In [ ]:
solveTaxi(q_table,True)

¿Crees que ha funcionado bien el aprendizaje? ¿Cómo puedes mejorarlo?

### Ejemplo 2: Frozen Lake
Vamos a ver un segundo ejemplo con el entorno FrozenLake
https://gymnasium.farama.org/environments/toy_text/frozen_lake/

En este entorno el agente controla el movimiento de un personaje en un mundo de rejilla (por defecto 4x4 pero se puede ampliar)
Algunas baldosas son transitables (walkable) y otras hacen que el agente caiga al agua. 

La dirección real de movimiento del agente es incierta y solo depende parcialmente de la dirección elegida (porque puede resbalar en el hielo, sólo si el parámetro de inicialización is_slippery=True). 
La recompensa se obtiene cuando el agente llega a traves de un camino transitable a una casilla objetivo.


In [ ]:
import gymnasium as gym
problema = gym.make("FrozenLake-v1", render_mode="human",map_name="4x4",is_slippery=False)
print("Tamaño de espacio de estados", problema.observation_space)
print("Estado aleatorio", problema.observation_space.sample())
size_estados = problema.observation_space.n
print("Hay", size_estados, " estados posibles.")
problema.reset()
problema.render()
env=problema

In [ ]:
# Realizando acciones aleatorias no se resuelve el problema :) puedes usar búsqueda o aprendizaje por refuerzo. 
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        #print(observation)
        action = env.action_space.sample()
        observation, reward, done, info,_  = env.step(action)
        
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

### Actividad

Realiza el entrenamiento para el agente Frozen Lake utilizando el código anterior y observa si mejora su comportamiento. Como has observado en el código anterior, en este caso sí funciona la visualización en modo "human" aunque el render() hace más lento el entrenamiento. Puedes probarlo, pero te recomiendo que no lo uses para entrenar. 

### Ejemplo 3. CliffWalking  (para entregar) 

Para esta práctica se pide entrenar y analizar en detalle el comportamiento de un agente para el problema CliffWalking (y/o BlackJack).
Puedes encontrar más información en la documentación de Gymnasium:
[https://gymnasium.farama.org/environments/toy_text/cliff_walking/]

El juego comienza con el jugador en la ubicación (3, 0) de una cuadrícula de 4x12, con el objetivo situado en (3, 11). Si el jugador llega al objetivo, el episodio termina.
Un precipicio se extiende a lo largo de (3, 1..10). Si el jugador se mueve a una ubicación del precipicio, vuelve al punto de inicio.

El jugador realiza movimientos hasta llegar al objetivo.

Adaptado del Ejemplo 6.6 (página 132) del libro Reinforcement Learning: An Introduction de Sutton y Barto.
Según el problema original (y la documentación) el precipicio puede configurarse como resbaladizo (desactivado por defecto), por lo que el jugador puede moverse perpendicularmente a la dirección deseada en ocasiones (ver is_slippery). Sin embargo, no está disponible la opción en la versión v0 proporcionada por gymnasium, por lo que no usaremos la opción.

In [4]:
import gymnasium as gym
problema = gym.make("CliffWalking-v0", render_mode="ansi")# "human", "rgb_array", or "ansi"

In [5]:
print("Tamaño de espacio de estados", problema.observation_space)
print("Estado aleatorio", problema.observation_space.sample())
size_estados = problema.observation_space.n
print("Hay", size_estados, " estados posibles.")
problema.reset()
#problema.render()
env=problema

Tamaño de espacio de estados Discrete(48)
Estado aleatorio 2
Hay 48  estados posibles.


In [7]:
for i_episode in range(20):
    observation = env.reset()
    for t in range(1000):
        env.render()
        #print(observation)
        action = env.action_space.sample()
        observation, reward, done, info,_  = env.step(action)
        
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

KeyboardInterrupt: 

¿Qué tal funciona la resolución con acciones aleatorias? 

Entrena un agente que resuelva el problema y comenta en detalle los resultados modificando los valores de los párametros, el número de episodios de aprendizaje, la recompensa por defecto. Define una métrica adecuada para el problema y evalua el aprendizaje para cada caso. 
Indica al final cual es la configuración de parámetros elegida. Al final del notebook se dan más detalles en **Actividad para Entregar**.

In [ ]:

import numpy as np

#Codigo de arriba

q_table = np.zeros([env.observation_space.n, env.action_space.n])

"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters

alpha = 0.1 #moderadamente alto para ser el factor de aprendizaje, irá decayendo hasta 0.01
gamma = 0.9 #en un entorno inicialmente desconocido nos interesa el aprendizaje a largo plazo
epsilon = 1.0 #Al principio la exploración debe ser muy alta e irá decayendo hasta 0
epsilon_min = 0
epsilon_decay = 0.999
alpha_min = 0.01
alpha_decay = 0.9999
episodes = 100000

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, episodes+1):
    state,_ = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info, _ = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0: #al principio para depurar que se muestren todos
        clear_output(wait=True)
        print(f"Episode: {i}")
        print(f"Alpha: {alpha}, Epsilon: {epsilon}, Gamma {gamma}")
        print(f"q_matrix:\n: {q_table}")

print(q_table)
print("Training finished.\n")

Episode: 4000
Alpha: 0.1, Epsilon: 1.0, Gamma 0.9
q_matrix:
: [[  -7.94108868   -7.71232075   -7.71232075   -7.94108868]
 [  -7.71232075   -7.45813417   -7.45813417   -7.94108868]
 [  -7.45813417   -7.17570464   -7.17570464   -7.71232075]
 [  -7.17570464   -6.86189404   -6.86189404   -7.45813417]
 [  -6.86189404   -6.5132156    -6.5132156    -7.17570464]
 [  -6.5132156    -6.12579511   -6.12579511   -6.86189404]
 [  -6.12579511   -5.6953279    -5.6953279    -6.5132156 ]
 [  -5.6953279    -5.217031     -5.217031     -6.12579511]
 [  -5.217031     -4.68559      -4.68559      -5.6953279 ]
 [  -4.68559      -4.0951       -4.0951       -5.217031  ]
 [  -4.0951       -3.439        -3.439        -4.68559   ]
 [  -3.439        -3.439        -2.71         -4.0951    ]
 [  -7.94108868   -7.45813417   -7.45813417   -7.71232075]
 [  -7.71232075   -7.17570464   -7.17570464   -7.71232075]
 [  -7.45813417   -6.86189404   -6.86189404   -7.45813417]
 [  -7.17570464   -6.5132156    -6.5132156    -7.1757

### Ejemplo 4. Blackjack  (para entregar) 

Entrenar un agente para el problema Blackjack. 
Puedes encontrar más información en la documentación de Gymnasium:
[https://gymnasium.farama.org/environments/toy_text/blackjack/]

Blackjack es un juego de cartas en el que el objetivo es vencer al crupier obteniendo cartas que sumen más cerca de 21 (sin exceder 21) que las cartas del crupier. El juego comienza con el crupier teniendo una carta boca arriba y otra boca abajo, mientras que el jugador tiene dos cartas boca arriba. Todas las cartas se extraen de un mazo infinito (es decir, con reposición).
Los valores de las cartas son los siguientes:
    Las figuras (Jota, Reina, Rey) tienen un valor de 10 puntos.
    Los ases pueden contarse como 11 (llamado "as usable") o como 1.
    Las cartas numéricas (del 2 al 9) tienen un valor igual a su número.
    El jugador tiene la suma de las cartas en su poder. Puede solicitar cartas adicionales (pedir) hasta que decida detenerse (plantarse) o exceda 21 (lo que resulta en pérdida inmediata, llamada quiebra).

Después de que el jugador se planta, el crupier revela su carta oculta y roba cartas hasta que su suma sea 17 o mayor. Si el crupier se pasa de 21, el jugador gana.
Si ni el jugador ni el crupier se pasan, el resultado (ganar, perder, empatar) se decide por quién se acerca más a 21.

Este entorno corresponde a la versión del problema de blackjack descrita en el Ejemplo 5.1 del libre _Reinforcement Learning: An Introduction de Sutton y Barto_.

Existen dos versiones del juego que se configuran con los parámetros al crear el entorno:

**natural=False**: indica si se debe otorgar una recompensa adicional por iniciar con un blackjack natural, es decir, comenzar con un as y un diez (suma total de 21).
    
**sab = False**: i Indica si se deben seguir exactamente las reglas descritas en el libro de Sutton y Barto. Si sab es True, el argumento natural será ignorado. Si el jugador logra un blackjack natural y el crupier no, el jugador ganará (recibiendo una recompensa de +1). La regla inversa no se aplica. Si tanto el jugador como el crupier obtienen un blackjack natural, el resultado será un empate (recompensa de 0   ).

### Ejemplo 4. Blackjack  (para entregar) 

Entrenar un agente para el problema Blackjack. 
Puedes encontrar más información en la documentación de Gymnasium:
[https://gymnasium.farama.org/environments/toy_text/blackjack/]


In [8]:
### Ejemplo 4. Blackjack

import gymnasium as gym
env=gym.make('Blackjack-v1', natural=False, sab=False, render_mode="human")

In [9]:
for i_episode in range(20):
    observation = env.reset()
    for t in range(1000):
        env.render()
        #print(observation)
        action = env.action_space.sample()
        observation, reward, done, info,_  = env.step(action)
        
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

Episode finished after 2 timesteps
Episode finished after 1 timesteps
Episode finished after 1 timesteps
Episode finished after 1 timesteps
Episode finished after 2 timesteps
Episode finished after 1 timesteps
Episode finished after 1 timesteps
Episode finished after 1 timesteps
Episode finished after 1 timesteps
Episode finished after 1 timesteps
Episode finished after 2 timesteps
Episode finished after 1 timesteps
Episode finished after 1 timesteps
Episode finished after 1 timesteps


KeyboardInterrupt: 

## Actividad para entregar

Configura el aprendizaje por refuerzo para resolver alguno/s de los problemas Toy Text de Gymnasium utilizando distintos valores de configuración de los parámetros y observa qué configuración se comporta mejor.
     
    1. Define las métricas adecuadas para poder evaluar qué tal se comporta el agente entrenado con distintas configuraciones de parámetros.
    2. Realiza una variación dinámica de los valores de los parámetros durante el entrenamiento para que los valores no sean fijos 
    2. Modifica la recompensa por defecto para ver cómo afecta al entrenamiento. Observa cuál es la función de recompensa que se define por defecto (consulta en la documentación de gym). Mejórala reescribiendo el valor de reward y observa cómo afecta la mejora de la función de recompensa en el proceso de aprendizaje. Explica cómo has medido esta mejora.
    4. Realiza pruebas y saca conclusiones claras justificadas con el resumen de resultados. Puedes usar gráficas.  

    

### Actividad opcional 

Piensa si podrías facilmente resolver el problema del taxi con cualquiera de los algoritmos de búsqueda de AIMA. 
Sólo tendrías que definir una subclase de Problem en la que los estados utilicen la codificación numérica de los 500 estados del entorno env de Gym con las 6 acciones.
    - 0: move south
    - 1: move north
    - 2: move east
    - 3: move west
    - 4: pickup passenger
    - 5: drop off passenger

En el código de la clase Problem tendrás que hacer uso de las funciones de env, por ejemplo, para implementar la función result que aplica una acción en un estado tendrías que hacer una llamada a env.step:  

    env.next_state, reward, done, info, X = env.step(action)

Aunque no realices la implementación de la clase para resolver el problema con búsqueda, piensa cómo se comportaría la búsqueda en espacio de estados ¿resolvería el problema de forma sencilla? ¿lo resolvería mejor que con Q-Learning?